# Title: LiveEO case study - "finding Capella (SAR) image pairs"
#### Date: 16.09.2025
#### Author: Emma Underwood
##### Workflow type: Pre-processing
##### Aim of workflow: Filtering image pairs to become training samples for a change detection model
##### Inputs: Parquet files: 1 x SAR images set (Capella), 1 x SAR images metadata
##### Outputs: Summary statistics, methods and findings report (html), list of Capella pairs (as parquet, and human-friendly csv), web map of their distribution (html)

## STEP 1: Setting up

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Step 1: Import all tools required for workflow
"""

import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Core data handling - prioritize DuckDB for large data
import duckdb
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
from shapely.geometry import shape, mapping

# Spatial operations
from pyproj import CRS, Transformer

# visualisation
import matplotlib.pyplot as plt
import folium
from IPython.display import display, HTML, Markdown

# Date handling
from datetime import datetime, timedelta

# Utilities
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
from tqdm import tqdm

# Configure pandas for display only
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)
pd.options.mode.copy_on_write = True

# Single DuckDB connection for entire workflow
conn = duckdb.connect()
conn.execute("INSTALL spatial; LOAD spatial;")

print("✓ All imports loaded")
print("✓ DuckDB spatial extension loaded")
print("  ... Systems ready!")

✓ All imports loaded
✓ DuckDB spatial extension loaded
  ... Systems ready!


In [2]:
"""
Set up project directory structure and configuration parameters
Create organised folder structure, define file paths, processing parameters
"""

# Local git location - all relative paths after here
WORKING_DIR = Path(r"C:\Users\Emma Underwood\Documents\GitHub\LiveEO")
#WORKING_DIR = Path.cwd()  # Current directory

def setup_project_structure(base_path):
    """Create project directory structure within git repo"""
    directories = [
        "data/raw",                 # Original parquet files - add to .gitignore
        "data/output",              # Final outputs (pairs files)
        "data/output/derived",      # Processed intermediate files
        "reports",                  # HTML/PDF reports
        "logs"                      # Processing logs
    ]
    
    # Create all directories relative to base path
    for dir_path in directories:
        (base_path / dir_path).mkdir(parents=True, exist_ok=True)
    
    # Create .gitignore in project root for the big files
    gitignore_path = base_path / '.gitignore'
    gitignore_content = """
# Large data files - don't commit!
data/raw/*.parquet
data/raw/*.tif  
data/raw/*.zip

# Temporary processing files
*.tmp
*.log
.ipynb_checkpoints/

# OS generated files
.DS_Store
Thumbs.db
"""
    
    if not gitignore_path.exists():
        with open(gitignore_path, 'w') as f:
            f.write(gitignore_content)
    
    print(f"Project structure created in: {base_path}")
    return True

# Configuration class - all paths relative to working directory
class Config:
    """Configuration parameters for satellite imagery processing"""
    
    # Paths
    PROJECT_ROOT = WORKING_DIR
    RAW_DATA_PATH = PROJECT_ROOT / "data/raw"
    OUTPUT_PATH = PROJECT_ROOT / "data/output"
    
    # Data files
    CAPELLA_ARCHIVE_PATH = RAW_DATA_PATH / "capella_archive.parquet"
    CAPELLA_STAC_PATH = RAW_DATA_PATH / "capella_stac.parquet"
    #SKYSAT_PATH = RAW_DATA_PATH / "skysat.parquet"
    
    # Processing parameters
    TEMPORAL_WINDOW_MIN = 7  # minimum days between SAR acquisitions
    TEMPORAL_WINDOW_MAX = 14  # maximum days between SAR acquisitions
    OVERLAP_THRESHOLD = 95.0  # % spatial overlap required
    
    # Angle thresholds for different quality levels
    ANGLE_THRESHOLDS = {
        'strict': 1,
        'moderate': 2.0,
        'relaxed': 5.0
    }

# Create directories
for dir_name in ["data/raw", "data/output", "figures"]:
    (WORKING_DIR / dir_name).mkdir(parents=True, exist_ok=True)

config = Config()

# Quick file check
missing_files = []
for path in [config.CAPELLA_ARCHIVE_PATH, config.CAPELLA_STAC_PATH]:
    if not path.exists():
        missing_files.append(str(path))

if missing_files:
    print(f"WARNING: Missing files: {missing_files}")
else:
    print("✓ All required files found")

✓ All required files found


In [3]:
"""
Define helper/utility functions
"""

def create_processing_log(operation: str, start_time: datetime, 
                         records_in: int, records_out: int) -> Dict:
    """Create standardised log entry for processing operations"""
    duration = (datetime.now() - start_time).total_seconds()
    
    return {
        'operation': operation,
        'duration_seconds': duration,
        'records_in': records_in,
        'records_out': records_out,
        'processing_rate': records_in / duration if duration > 0 else 0
    }

print("✓ Helper functions defined")

✓ Helper functions defined


# Objective 1: Exploring SAR data

## (First quick look)

In [4]:
# What are the data columns in the SAR parquet
# What columns/field names should be in the analysis etc.
import pandas as pd
pd.read_parquet(config.CAPELLA_ARCHIVE_PATH).head(1)


,id,datetime,mode,products,url,look_angle,incd_angle,sqnt_angle,num_looks,observ_dir,...,orbt_plane,polariztn,slc,geo,gec,sicd,sidd,cphd,geom,geometry_converted
0,383ebc02-264f-4ec9-851a-352545199c0a,2025-08-24 04:51:09.100659+00:00,spotlight,"SLC, GEO, GEC, SICD, SIDD",https://console.capellaspace.com/search/?q=383...,18.5,20.4,0.0,1,right,...,53,HH,CAPELLA_C11_SP_SLC_HH_20250824045104_202508240...,CAPELLA_C11_SP_GEO_HH_20250824045104_202508240...,CAPELLA_C11_SP_GEC_HH_20250824045104_202508240...,CAPELLA_C11_SP_SICD_HH_20250824045104_20250824...,CAPELLA_C11_SP_SIDD_HH_20250824045104_20250824...,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,0103000020E610000001000000050000002B297F102C30...


## STEP 2: data loading, exploring

In [5]:
"""
Step 2: Data exploration using DuckDB and exporting csv
"""

def quick_explore_data(conn: duckdb.DuckDBPyConnection, config: Config) -> Dict:
    """Quick exploration of data structure with export to file"""
    
    print("="*60)
    print("DATA EXPLORATION")
    print("="*60)
    
    results = {}
    summary_rows = []
    
    # Check Capella Archive
    if config.CAPELLA_ARCHIVE_PATH.exists():
        # Basic statistics
        query = f"""
        SELECT 
            COUNT(*) as total_rows,
            COUNT(DISTINCT id) as unique_images,
            MIN(datetime) as earliest_date,
            MAX(datetime) as latest_date,
            COUNT(DISTINCT polariztn) as polarizations,
            COUNT(DISTINCT mode) as modes,
            AVG(look_angle) as avg_look_angle,
            MIN(look_angle) as min_look_angle,
            MAX(look_angle) as max_look_angle
        FROM '{config.CAPELLA_ARCHIVE_PATH}'
        """
        
        stats = conn.execute(query).fetchone()
        
        results['capella_archive'] = {
            'total_rows': stats[0],
            'unique_images': stats[1],
            'date_range': (stats[2], stats[3]),
            'polarizations': stats[4],
            'modes': stats[5],
            'look_angle_stats': (stats[6], stats[7], stats[8])
        }
        
        print(f"Capella Archive:")
        print(f"  {stats[0]:,} images")
        print(f"  Date range: {stats[2]:%Y-%m-%d} to {stats[3]:%Y-%m-%d}")
        print(f"  {stats[4]} polarizations, {stats[5]} acquisition modes")
        print(f"  Look angles: {stats[7]:.1f}° to {stats[8]:.1f}° (avg: {stats[6]:.1f}°)")
        
        # Add to summary table
        summary_rows.append({
            'Dataset': 'Capella Archive',
            'Total_Records': stats[0],
            'Unique_Images': stats[1],
            'Start_Date': stats[2].strftime('%Y-%m-%d') if stats[2] else 'N/A',
            'End_Date': stats[3].strftime('%Y-%m-%d') if stats[3] else 'N/A',
            'Polarizations': stats[4],
            'Modes': stats[5],
            'Avg_Look_Angle': f"{stats[6]:.1f}" if stats[6] else 'N/A'
        })
        
        # Get polarization breakdown
        pol_query = f"""
        SELECT polariztn, COUNT(*) as count
        FROM '{config.CAPELLA_ARCHIVE_PATH}'
        GROUP BY polariztn
        ORDER BY count DESC
        """
        pol_results = conn.execute(pol_query).fetchall()
        
        print("\n  Polarization distribution:")
        for pol, count in pol_results:
            print(f"    {pol}: {count:,} ({count/stats[0]*100:.1f}%)")
            
        # Get mode breakdown
        mode_query = f"""
        SELECT mode, COUNT(*) as count
        FROM '{config.CAPELLA_ARCHIVE_PATH}'
        GROUP BY mode
        ORDER BY count DESC
        """
        mode_results = conn.execute(mode_query).fetchall()
        
        print("\n  Acquisition mode distribution:")
        for mode, count in mode_results:
            print(f"    {mode}: {count:,} ({count/stats[0]*100:.1f}%)")
        
        # Get temporal distribution by month
        temporal_query = f"""
        SELECT 
            DATE_TRUNC('month', datetime) as month,
            COUNT(*) as count
        FROM '{config.CAPELLA_ARCHIVE_PATH}'
        WHERE datetime IS NOT NULL
        GROUP BY month
        ORDER BY month
        """
        temporal_results = conn.execute(temporal_query).fetchall()
        
        # Store additional details
        results['capella_archive']['polarization_dist'] = dict(pol_results)
        results['capella_archive']['mode_dist'] = dict(mode_results)
        results['capella_archive']['temporal_dist'] = [(str(m), c) for m, c in temporal_results]
        
        # Get column information
        columns = conn.execute(f"DESCRIBE SELECT * FROM '{config.CAPELLA_ARCHIVE_PATH}' LIMIT 0").fetchall()
        print(f"\n  {len(columns)} columns available")
        results['capella_archive']['columns'] = [col[0] for col in columns]
    
    # Check STAC metadata if exists
    if config.CAPELLA_STAC_PATH.exists():
        query = f"""
        SELECT COUNT(*) as total_rows
        FROM '{config.CAPELLA_STAC_PATH}'
        """
        stac_count = conn.execute(query).fetchone()[0]
        
        summary_rows.append({
            'Dataset': 'Capella STAC',
            'Total_Records': stac_count,
            'Unique_Images': 'N/A',
            'Start_Date': 'N/A',
            'End_Date': 'N/A',
            'Polarizations': 'N/A',
            'Modes': 'N/A',
            'Avg_Look_Angle': 'N/A'
        })
        
        print(f"\nCapella STAC: {stac_count:,} records")
    
    # Export exploration results to files
    print("\n" + "-"*60)
    print("Exporting exploration results...")
    
    # 1. Export summary table as CSV
    summary_df = pd.DataFrame(summary_rows)
    csv_path = config.OUTPUT_PATH / 'data_exploration_summary.csv'
    summary_df.to_csv(csv_path, index=False)
    print(f"  Summary CSV: {csv_path.name}")
    
    # 2. Export detailed results as JSON
    json_path = config.OUTPUT_PATH / 'data_exploration_detailed.json'
    with open(json_path, 'w') as f:
        # Convert datetime objects to strings for JSON serialization
        json_results = json.loads(pd.Series(results).to_json())
        json.dump(json_results, f, indent=2, default=str)
    print(f"  Detailed JSON: {json_path.name}")
    
    # 3. Export as formatted HTML report
    html_content = f"""
    <html>
    <head>
        <title>SAR Data Exploration Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h1 {{ color: #333; }}
            h2 {{ color: #666; border-bottom: 1px solid #ccc; padding-bottom: 5px; }}
            table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
            th {{ background-color: #f0f0f0; padding: 10px; text-align: left; border: 1px solid #ddd; }}
            td {{ padding: 8px; border: 1px solid #ddd; }}
            .metadata {{ background-color: #f9f9f9; padding: 10px; margin: 10px 0; border-radius: 5px; }}
        </style>
    </head>
    <body>
        <h1>SAR Data Exploration Report</h1>
        <div class="metadata">
            <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p><strong>Data Path:</strong> {config.RAW_DATA_PATH}</p>
        </div>
        
        <h2>Dataset Summary</h2>
        {summary_df.to_html(index=False, table_id='summary-table')}
    """
    
    # Add detailed breakdowns if available
    if 'capella_archive' in results:
        ca = results['capella_archive']
        
        # Polarization distribution
        if 'polarization_dist' in ca:
            pol_df = pd.DataFrame(list(ca['polarization_dist'].items()), 
                                 columns=['Polarization', 'Count'])
            pol_df['Percentage'] = (pol_df['Count'] / pol_df['Count'].sum() * 100).round(1)
            
            html_content += f"""
        <h2>Polarization Distribution</h2>
        {pol_df.to_html(index=False)}
        """
        
        # Mode distribution
        if 'mode_dist' in ca:
            mode_df = pd.DataFrame(list(ca['mode_dist'].items()), 
                                  columns=['Acquisition Mode', 'Count'])
            mode_df['Percentage'] = (mode_df['Count'] / mode_df['Count'].sum() * 100).round(1)
            
            html_content += f"""
        <h2>Acquisition Mode Distribution</h2>
        {mode_df.to_html(index=False)}
        """
        
        # Temporal distribution (first 12 months)
        if 'temporal_dist' in ca and len(ca['temporal_dist']) > 0:
            temp_df = pd.DataFrame(ca['temporal_dist'][:12], 
                                  columns=['Month', 'Image Count'])
            
            html_content += f"""
        <h2>Temporal Distribution (Recent Months)</h2>
        {temp_df.to_html(index=False)}
        """
        
        # Column list
        if 'columns' in ca:
            col_list = ', '.join(ca['columns'][:20])  # First 20 columns
            if len(ca['columns']) > 20:
                col_list += f", ... ({len(ca['columns'])-20} more)"
            
            html_content += f"""
        <h2>Available Columns</h2>
        <div class="metadata">
            <p><strong>Total columns:</strong> {len(ca['columns'])}</p>
            <p><strong>Column names:</strong> {col_list}</p>
        </div>
        """
    
    html_content += """
    </body>
    </html>
    """
    
    html_path = config.OUTPUT_PATH / 'data_exploration_report.html'
    with open(html_path, 'w') as f:
        f.write(html_content)
    print(f"  HTML report: {html_path.name}")
    
    print("\nExploration complete and exported")
    
    return results

# Run exploration with export
exploration_results = quick_explore_data(conn, config)

DATA EXPLORATION
Capella Archive:
  124,958 images
  Date range: 2020-10-19 to 2025-08-25
  2 polarizations, 3 acquisition modes
  Look angles: 3.2° to 58.1° (avg: 33.9°)

  Polarization distribution:
    HH: 118,503 (94.8%)
    VV: 6,455 (5.2%)

  Acquisition mode distribution:
    spotlight: 93,427 (74.8%)
    stripmap: 21,722 (17.4%)
    sliding_spotlight: 9,809 (7.8%)

  21 columns available

Capella STAC: 124,449 records

------------------------------------------------------------
Exporting exploration results...
  Summary CSV: data_exploration_summary.csv
  Detailed JSON: data_exploration_detailed.json
  HTML report: data_exploration_report.html

Exploration complete and exported


# Objective 2: Assemble Capella pairs

### [Intermediate step to debug the 'geom' format]

In [6]:
# """
# Debug geometry format to understand how to parse the "geom" column
# Can we use DuckDB?
# """
# # Check if DuckDB spatial functions work on this data
# print("\n" + "=" * 60)
# print("TESTING DUCKDB SPATIAL APPROACH")
# print("=" * 60)

# def test_duckdb_spatial_operations():
#     """
#     Test if we can do spatial operations directly in DuckDB
#     This might be more efficient than converting to shapely
#     """
    
#     file_path = config.CAPELLA_ARCHIVE_PATH
    
#     try:
#         # Test basic spatial functions
#         query = f"""
#         SELECT 
#             COUNT(*) as total_geoms,
#             COUNT(CASE WHEN ST_IsValid(geom) THEN 1 END) as valid_geoms,
#             ST_GeometryType(geom) as geom_type,
#             COUNT(*)
#         FROM '{file_path}'
#         WHERE geom IS NOT NULL
#         GROUP BY ST_GeometryType(geom)
#         LIMIT 5
#         """
        
#         spatial_stats = conn.execute(query).fetchall()
        
#         print("Spatial statistics:")
#         for row in spatial_stats:
#             print(f"  {row[2]}: {row[3]:,} geometries, {row[1]:,} valid")
        
#         # Test spatial bounds calculation
#         query_bounds = f"""
#         SELECT 
#             MIN(ST_XMin(geom)) as min_x,
#             MAX(ST_XMax(geom)) as max_x,
#             MIN(ST_YMin(geom)) as min_y,
#             MAX(ST_YMax(geom)) as max_y
#         FROM '{file_path}'
#         WHERE geom IS NOT NULL
#         LIMIT 1000
#         """
        
#         bounds_result = conn.execute(query_bounds).fetchone()
        
#         if bounds_result:
#             print(f"\nSpatial extent (sample):")
#             print(f"  Longitude: {bounds_result[0]:.6f} to {bounds_result[1]:.6f}")
#             print(f"  Latitude: {bounds_result[2]:.6f} to {bounds_result[3]:.6f}")
        
#         # Test overlap calculation between two geometries
#         query_overlap_test = f"""
#         WITH test_pairs AS (
#             SELECT 
#                 a.id as id1,
#                 b.id as id2,
#                 a.geom as geom1,
#                 b.geom as geom2,
#                 a.datetime as date1,
#                 b.datetime as date2
#             FROM '{file_path}' a
#             JOIN '{file_path}' b ON (
#                 b.datetime > a.datetime 
#                 AND (b.datetime::DATE - a.datetime::DATE) BETWEEN 7 AND 14
#             )
#             WHERE a.geom IS NOT NULL AND b.geom IS NOT NULL
#             LIMIT 5
#         )
#         SELECT 
#             id1,
#             id2,
#             ST_Area(ST_Intersection(geom1, geom2)) as intersection_area,
#             ST_Area(geom1) as area1,
#             ST_Area(geom2) as area2,
#             (ST_Area(ST_Intersection(geom1, geom2)) / LEAST(ST_Area(geom1), ST_Area(geom2))) * 100 as overlap_pct
#         FROM test_pairs
#         WHERE ST_Intersects(geom1, geom2)
#         """
        
#         overlap_results = conn.execute(query_overlap_test).fetchall()
        
#         print(f"\nTesting spatial overlaps on {len(overlap_results)} intersecting pairs:")
#         for row in overlap_results:
#             print(f"  Pair {row[0]}-{row[1]}: {row[5]:.1f}% overlap")
        
#         return len(overlap_results) > 0
        
#     except Exception as e:
#         print(f"DuckDB spatial operations failed: {e}")
#         return False

# # Test DuckDB spatial approach
# duckdb_spatial_works = test_duckdb_spatial_operations()

# if duckdb_spatial_works:
#     print(f"\nSOLUTION FOUND: Use DuckDB spatial functions directly!")
#     print("The geometry format works with DuckDB's built-in spatial operations.")
#     print("We should revise the pair matching to use SQL spatial functions instead of shapely.")
# else:
#     print(f"\nNeed to investigate further - DuckDB spatial functions also failed.")
    
#     # Additional debugging - check if it's a known format
#     print("\nTrying additional geometry format detection...")
    
#     # Check first few bytes for format signatures
#     sample_bytes = samples[0][2][:20] if samples else None
#     if sample_bytes:
#         hex_start = sample_bytes[:8].hex()
#         print(f"First 8 bytes (hex): {hex_start}")
        
#         # Common geometry format signatures
#         format_signatures = {
#             '01': 'Little-endian WKB',
#             '00': 'Big-endian WKB', 
#             '0001': 'Point (little-endian)',
#             '0002': 'LineString (little-endian)',
#             '0003': 'Polygon (little-endian)',
#             '2000': 'SRID prefix (big-endian)',
#             '0020': 'SRID prefix (little-endian)'
#         }
        
#         for sig, desc in format_signatures.items():
#             if hex_start.startswith(sig.lower()):
#                 print(f"Possible format: {desc}")
#                 break

## STEP 3: Capella pairs matching

In [ ]:
"""
Step 3: SAR pair matching using DuckDB + spatial operations
Runs everything in DuckDB, no intermediate files, no pandas registration.
"""

class SARPairMatcher:
    """SAR pair matching with better performance using DuckDB"""

    def __init__(self, config: Config, conn: duckdb.DuckDBPyConnection):
        self.config = config
        self.conn = conn

    def find_all_pairs(self, angle_threshold: float = 5.0,
                      sample_limit: int = None) -> pd.DataFrame:
        """
        Find all valid SAR pairs in one stepped query
        Uses DuckDB's spatial functions for overlap calculation
        Preserves geometry columns for validation purposes.
        """

        print(f"\nFinding pairs with ≤{angle_threshold}° angle difference...")

        file_path = self.config.CAPELLA_ARCHIVE_PATH
        limit_clause = f"LIMIT {sample_limit}" if sample_limit else ""

        query = f"""
        WITH sar_data AS (
            SELECT 
                id, datetime, geom, polariztn, mode, look_angle,
                datetime::DATE as date_only
            FROM '{file_path}'
            WHERE geom IS NOT NULL 
                AND datetime IS NOT NULL 
                AND polariztn IS NOT NULL 
                AND mode IS NOT NULL 
                AND look_angle IS NOT NULL
            {limit_clause}
        ),
        candidate_pairs AS (
            SELECT 
                a.id as id1,
                b.id as id2,
                a.datetime as date1,
                b.datetime as date2,
                EXTRACT(EPOCH FROM (b.datetime - a.datetime))/86400 as temporal_baseline_days,
                a.polariztn as polarisation,
                a.mode as mode,
                a.look_angle as angle1,
                b.look_angle as angle2,
                ABS(a.look_angle - b.look_angle) as angle_diff,
                a.geom as geom1,
                b.geom as geom2
            FROM sar_data a
            INNER JOIN sar_data b ON (
                b.date_only > a.date_only
                AND b.date_only <= a.date_only + INTERVAL '14 days'
                AND b.date_only >= a.date_only + INTERVAL '7 days'
                AND a.polariztn = b.polariztn
                AND a.mode = b.mode
                AND ABS(a.look_angle - b.look_angle) <= {angle_threshold}
            )
        ),
        spatial_pairs AS (
            SELECT 
                *,
                ST_Area(ST_Intersection(geom1, geom2)) / 
                    LEAST(ST_Area(geom1), ST_Area(geom2)) * 100 as overlap_pct
            FROM candidate_pairs
            WHERE ST_Intersects(geom1, geom2)
        )
        SELECT 
            id1, id2, date1, date2, 
            temporal_baseline_days,
            polarisation, mode,
            angle1, angle2, angle_diff,
            overlap_pct,
            geom1, geom2,
            ST_AsText(geom1) as wkt_geom1,
            ST_AsText(geom2) as wkt_geom2
        FROM spatial_pairs
        WHERE overlap_pct >= {self.config.OVERLAP_THRESHOLD}
        ORDER BY overlap_pct DESC, angle_diff ASC
        """

        try:
            result_df = self.conn.execute(query).fetch_df()
            print(f"  Found {len(result_df)} valid pairs")

            if len(result_df) > 0:
                print(f"  Mean overlap: {result_df['overlap_pct'].mean():.1f}%")
                print(f"  Mean angle diff: {result_df['angle_diff'].mean():.2f}°")

            return result_df

        except Exception as e:
            print(f"  Error: {e}")
            return pd.DataFrame()

    def process_multiple_thresholds(self, thresholds=None) -> dict:
        """
        Process pairs for selected angle thresholds.
        Pass a list of keys (e.g., ['strict']) to process only those thresholds.
        If None, process all defined in config.ANGLE_THRESHOLDS.
        """

        print("="*70)
        print("SAR PAIR MATCHING")
        print("="*70)

        results = {}

        # Determine which thresholds to run
        threshold_dict = self.config.ANGLE_THRESHOLDS
        if thresholds is None:
            keys_to_run = list(threshold_dict.keys())
        else:
            keys_to_run = [k for k in thresholds if k in threshold_dict]

        for name in keys_to_run:
            threshold = threshold_dict[name]
            print(f"\n{name.upper()} threshold ({threshold}°):")
            pairs_df = self.find_all_pairs(angle_threshold=threshold)

            if not pairs_df.empty:
                output_path = self.config.OUTPUT_PATH / f'pairs_{name}_{threshold}deg.parquet'
                pairs_df.to_parquet(output_path, index=False)
                results[name] = pairs_df

                print(f"  Saved to: {output_path.name}")

        # Summary
        print("\n" + "="*70)
        print("SUMMARY")
        print("="*70)
        for name, df in results.items():
            if not df.empty:
                print(f"{name}: {len(df)} pairs (overlap: {df['overlap_pct'].mean():.1f}±{df['overlap_pct'].std():.1f}%)")

        return results

# Example usage:
matcher = SARPairMatcher(config, conn)
strict_results = matcher.process_multiple_thresholds(thresholds=['strict'])

## Step 4: Validation (remove dups)

In [27]:
"""
Step 4: validation, dups removal
"""
# Last checks using panda and WKT cols for validation 
import pandas as pd

strict_file_path = config.OUTPUT_PATH / "pairs_strict_1deg.parquet"
df = pd.read_parquet(strict_file_path)

print(df.columns)

# Drop duplicates based on date1, date2, and identical geometry (WKT string)
dups = df.duplicated(subset=['date1', 'date2', 'wkt_geom1'], keep='first')
validated_df = df[~dups]

validated_path = strict_file_path.with_name(strict_file_path.stem + "_validated.parquet")
validated_df.to_parquet(validated_path, index=False)
print(f"Validated file saved to: {validated_path.name}")

Index(['id1', 'id2', 'date1', 'date2', 'temporal_baseline_days',
       'polarisation', 'mode', 'angle1', 'angle2', 'angle_diff', 'overlap_pct',
       'geom1', 'geom2', 'wkt_geom1', 'wkt_geom2'],
      dtype='object')
Validated file saved to: pairs_strict_1deg_validated.parquet


## STEP 5: Mapping the results

In [38]:
"""
Step 5: Interactive map for "strict", validated chosen pairs (1 deg angle diff) 
with country borders and zoom controls
"""

import folium
from folium import plugins
import json
import pandas as pd
from shapely import wkt
from typing import Dict

class SARPairVisualiser:
    """
    Create interactive maps with country borders and zoom controls
    """
    
    def __init__(self, config: Config):
        self.config = config
        self.conn = duckdb.connect()
        self.conn.execute("INSTALL spatial; LOAD spatial;")
    
    def load_sar_pairs(self, pairs_file: str = None) -> pd.DataFrame:
        """Load SAR pairs - specifically the strictest threshold"""
        if pairs_file is None:
            # Default to strictest pairs file
            pairs_file = 'pairs_strict_1deg.parquet'  # Or change to your exact filename
            print(f"Loading strictest pairs: {pairs_file}")
        
        pairs_path = self.config.OUTPUT_PATH / pairs_file
        
        # If strict file doesn't exist, check for alternative names
        if not pairs_path.exists():
            alternative_names = [
                'pairs_strict_1deg.parquet',
                'training_pairs_strict_1deg.parquet'
            ]
            
            for alt_name in alternative_names:
                alt_path = self.config.OUTPUT_PATH / alt_name
                if alt_path.exists():
                    pairs_path = alt_path
                    pairs_file = alt_name
                    print(f"Found strict pairs file: {pairs_file}")
                    break
            
            # If still not found, error out
            if not pairs_path.exists():
                print(f"ERROR: Strict pairs file not found. Looking for: {pairs_file}")
                print(f"Available files in output directory:")
                for f in self.config.OUTPUT_PATH.glob('*pairs*.parquet'):
                    print(f"  - {f.name}")
                return pd.DataFrame()
        
        # Load the strict pairs
        try:
            pairs_df = pd.read_parquet(pairs_path)
            print(f"Loaded {len(pairs_df)} STRICT pairs from {pairs_file}")
            
            # Show quality metrics
            if 'overlap_pct' in pairs_df.columns:
                print(f"  Mean overlap: {pairs_df['overlap_pct'].mean():.1f}%")
            if 'angle_diff' in pairs_df.columns:
                print(f"  Mean angle diff: {pairs_df['angle_diff'].mean():.2f}°")
            
            return pairs_df
        except Exception as e:
            print(f"Error loading pairs file: {e}")
            return pd.DataFrame()
    
    def get_pair_geometries(self, pairs_df: pd.DataFrame, max_pairs: int = 50) -> pd.DataFrame:
        """Retrieve actual geometries for selected pairs"""
        if pairs_df.empty:
            return pd.DataFrame()
        
        selected_pairs = pairs_df.head(max_pairs)
        print(f"Retrieving geometries for top {len(selected_pairs)} strict pairs...")
        
        id1_list = selected_pairs['id1'].tolist()
        id2_list = selected_pairs['id2'].tolist()
        all_ids = list(set(id1_list + id2_list))
        
        id_str = "', '".join(all_ids)
        
        file_path = self.config.CAPELLA_ARCHIVE_PATH
        query = f"""
        SELECT 
            id,
            datetime,
            ST_AsText(geom) as wkt_geom,
            polariztn,
            mode,
            look_angle
        FROM '{file_path}'
        WHERE id IN ('{id_str}')
        """
        
        try:
            results = self.conn.execute(query).fetchall()
            geom_df = pd.DataFrame(results, columns=['id', 'datetime', 'wkt_geom', 
                                                   'polarisation', 'mode', 'look_angle'])
            
            print(f"Retrieved geometries for {len(geom_df)} images")
            return geom_df
            
        except Exception as e:
            print(f"Error retrieving geometries: {e}")
            return pd.DataFrame()
    
    def create_enhanced_map(self, pairs_df: pd.DataFrame, geom_df: pd.DataFrame, 
                          map_title: str = None) -> folium.Map:
        """
        Create interactive map with country borders and zoom controls
        """
        if map_title is None:
            map_title = f"Strict SAR Training Pairs (≤2° angle difference)"
            
        print(f"Creating  interactive map: {map_title}")
        
        if pairs_df.empty or geom_df.empty:
            print("No data to map")
            return None
        
        # Parse all geometries and calculate bounds
        all_bounds = []
        geom_dict = {}
        
        for _, row in geom_df.iterrows():
            try:
                geom = wkt.loads(row['wkt_geom'])
                geom_dict[row['id']] = geom
                bounds = geom.bounds
                all_bounds.extend([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
            except:
                continue
        
        if not all_bounds:
            print("No valid geometries for mapping")
            return None
        
        # Calculate center
        lats = [b[0] for b in all_bounds]
        lons = [b[1] for b in all_bounds]
        center_lat = sum(lats) / len(lats)
        center_lon = sum(lons) / len(lons)
        
        # Create base map
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=6,
            tiles=None,
            max_zoom=18,
            min_zoom=2
        )
        
        # Add base layers with country borders
        folium.TileLayer(
            tiles='OpenStreetMap',
            name='OpenStreetMap',
            overlay=False,
            control=True
        ).add_to(m)
        
        folium.TileLayer(
            tiles='CartoDB positron',
            name='Light Map',
            overlay=False,
            control=True
        ).add_to(m)
        
        folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='Satellite',
            overlay=False,
            control=True
        ).add_to(m)
        
        # Add country borders overlay
        folium.TileLayer(
            tiles='https://stamen-tiles.a.ssl.fastly.net/toner-lines/{z}/{x}/{y}.png',
            attr='Stamen',
            name='Country Borders',
            overlay=True,
            control=True,
            opacity=0.5
        ).add_to(m)
        
        # Create feature groups for pairs
        pair_groups = folium.FeatureGroup(name='Strict SAR Pairs')
        
        # Define colors - use distinct colors for better visibility
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
                 'cadetblue', 'darkgreen', 'darkblue', 'pink']
        
        # Store pair bounds for zoom functionality
        pair_bounds = []
        
        # Add pairs to map
        for idx, row in pairs_df.iterrows():
            pair_num = idx + 1
            color = colors[idx % len(colors)]
            
            if row['id1'] in geom_dict and row['id2'] in geom_dict:
                try:
                    geom1 = geom_dict[row['id1']]
                    geom2 = geom_dict[row['id2']]
                    
                    # Calculate pair bounds for zoom
                    combined_bounds = [
                        [min(geom1.bounds[1], geom2.bounds[1]), 
                         min(geom1.bounds[0], geom2.bounds[0])],
                        [max(geom1.bounds[3], geom2.bounds[3]), 
                         max(geom1.bounds[2], geom2.bounds[2])]
                    ]
                    pair_bounds.append((pair_num, combined_bounds))
                    
                    # Format popup info
                    temporal_baseline = row.get('temporal_baseline_days', 'N/A')
                    overlap = row.get('overlap_pct', 'N/A')
                    angle_diff = row.get('angle_diff', 'N/A')
                    
                    overlap_str = f"{overlap:.1f}%" if isinstance(overlap, (int, float)) else str(overlap)
                    angle_str = f"{angle_diff:.2f}°" if isinstance(angle_diff, (int, float)) else str(angle_diff)
                    
                    # popup with zoom button
                    popup_html = f"""
                    <div style='width: 250px'>
                        <h4>Strict Pair {pair_num}</h4>
                        <table style='width:100%'>
                            <tr><td><b>Temporal:</b></td><td>{temporal_baseline} days</td></tr>
                            <tr><td><b>Overlap:</b></td><td>{overlap_str}</td></tr>
                            <tr><td><b>Angle Diff:</b></td><td>{angle_str}</td></tr>
                            <tr><td><b>Polarisation:</b></td><td>{row.get('polarisation', 'N/A')}</td></tr>
                            <tr><td><b>Mode:</b></td><td>{row.get('mode', 'N/A')}</td></tr>
                        </table>
                        <button onclick='
                            var bounds = {combined_bounds};
                            window[Object.keys(window).find(key => key.startsWith("map_"))].fitBounds(bounds);
                        ' style='margin-top:10px; width:100%; padding:5px; background:#4CAF50; color:white; 
                                 border:none; border-radius:4px; cursor:pointer'>
                            Zoom to This Pair
                        </button>
                    </div>
                    """
                    
                    # Add footprints
                    folium.GeoJson(
                        geom1.__geo_interface__,
                        style_function=lambda x, c=color: {
                            'fillColor': c, 'color': c, 'weight': 2, 'fillOpacity': 0.3
                        },
                        popup=folium.Popup(popup_html, max_width=300),
                        tooltip=f"Strict Pair {pair_num} - Image 1"
                    ).add_to(pair_groups)
                    
                    folium.GeoJson(
                        geom2.__geo_interface__,
                        style_function=lambda x, c=color: {
                            'fillColor': c, 'color': c, 'weight': 2, 
                            'fillOpacity': 0.1, 'dashArray': '5,5'
                        },
                        tooltip=f"Strict Pair {pair_num} - Image 2"
                    ).add_to(pair_groups)
                    
                except Exception as e:
                    print(f"Error adding pair {pair_num}: {e}")
                    continue
        
        pair_groups.add_to(m)
        
        # Add zoom controls for all pairs (sidebar)
        zoom_control_html = """
        <div style='position: fixed; top: 70px; right: 10px; width: 150px; 
                    background: white; padding: 10px; border-radius: 5px; 
                    box-shadow: 0 0 15px rgba(0,0,0,0.2); z-index: 1000;
                    max-height: 400px; overflow-y: auto;'>
            <h4 style='margin: 0 0 10px 0; font-size: 14px;'>Quick Zoom</h4>
        """
        
        for pair_num, bounds in pair_bounds[:10]:  # Show first 10 pairs
            zoom_control_html += f"""
            <button onclick='
                var bounds = {bounds};
                window[Object.keys(window).find(key => key.startsWith("map_"))].fitBounds(bounds);
            ' style='width:100%; margin:2px 0; padding:5px; background:#f0f0f0; 
                     border:1px solid #ddd; border-radius:3px; cursor:pointer;
                     font-size:12px;'
                     onmouseover='this.style.background="#e0e0e0"' 
                     onmouseout='this.style.background="#f0f0f0"'>
                Pair {pair_num}
            </button>
            """
        
        zoom_control_html += "</div>"
        
        # Add the zoom control panel
        m.get_root().html.add_child(folium.Element(zoom_control_html))
        
        # Add other controls
        folium.LayerControl(position='topright').add_to(m)
        
        # Add fullscreen button
        plugins.Fullscreen(
            position='topleft',
            title='Fullscreen',
            title_cancel='Exit Fullscreen',
            force_separate_button=True
        ).add_to(m)
        
        # Add measurement tool
        plugins.MeasureControl(position='topleft').add_to(m)
        
        # Add minimap
        minimap = plugins.MiniMap(toggle_display=True, width=200, height=150)
        m.add_child(minimap)
        
        # Add title with strict indicator
        title_html = f'''
        <div style='position: fixed; top: 10px; left: 50%; transform: translateX(-50%); 
                    z-index: 1000; background: white; padding: 10px; border-radius: 5px;
                    box-shadow: 0 0 15px rgba(0,0,0,0.2);'>
            <h3 style='margin: 0; font-size: 16px;'>{map_title}</h3>
            <p style='margin: 5px 0 0 0; font-size: 12px; text-align: center;'>
                {len(pairs_df)} high-quality pairs | Solid = Image 1, Dashed = Image 2
            </p>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(title_html))
        
        return m
    
    def create_visualisations(self, max_pairs: int = 25) -> Dict:
        """Complete workflow to create visualisations for strict (<1 deg diff) pairs only"""
        print("="*60)
        print("STRICT SAR PAIR VISUALISATION")
        print("="*60)
        
        # Load strict pairs only
        pairs_df = self.load_sar_pairs()  # Will load strict pairs by default
        if pairs_df.empty:
            return {'error': 'No strict pairs data found'}
        
        # Get geometries
        geom_df = self.get_pair_geometries(pairs_df, max_pairs=max_pairs)
        if geom_df.empty:
            return {'error': 'No geometries retrieved'}
        
        # Create map with strict pairs
        interactive_map = self.create_enhanced_map(
            pairs_df.head(max_pairs), 
            geom_df,
            f"Strict SAR Training Pairs (Top {min(max_pairs, len(pairs_df))} of {len(pairs_df)} total)"
        )
        
        if interactive_map:
            # Use specific filename for strict pairs
            map_path = self.config.OUTPUT_PATH / 'strict_pairs_map_ELU_16092025.html'
            interactive_map.save(str(map_path))
            print(f"Saved strict (1 deg) pairs map: {map_path}")
            
            return {
                'map': interactive_map,
                'map_file': str(map_path),
                'pairs_mapped': min(max_pairs, len(pairs_df)),
                'total_strict_pairs': len(pairs_df)
            }
        else:
            return {'error': 'Failed to create map'}

# Create visualisations for STRICT pairs
visualiser = SARPairVisualiser(config)
viz_results = visualiser.create_visualisations(max_pairs=20)

if 'error' not in viz_results:
    print(f"\nStrict pairs (1deg) map features:")
    print(f"  • {viz_results['pairs_mapped']} of {viz_results.get('total_strict_pairs', 'N/A')} strict pairs visualised")
    print(f"  • Country borders overlay for geographic context")
    print(f"  • Quick zoom sidebar for navigation")
    print(f"  • Individual zoom buttons in popups")
    print(f"  • Multiple basemap options")
    print(f"  • All pairs meet ≤1° angle difference requirement")
    viz_results['map']
else:
    print(f"Visualisation failed: {viz_results['error']}")

STRICT SAR PAIR VISUALISATION
Loading strictest pairs: pairs_strict_1deg.parquet
Loaded 13385 STRICT pairs from pairs_strict_1deg.parquet
  Mean overlap: 99.0%
  Mean angle diff: 0.34°
Retrieving geometries for top 20 strict pairs...
Retrieved geometries for 37 images
Creating  interactive map: Strict SAR Training Pairs (Top 20 of 13385 total)
Saved strict (1 deg) pairs map: C:\Users\Emma Underwood\Documents\GitHub\LiveEO\data\output\strict_pairs_map_ELU_16092025.html

Strict pairs (1deg) map features:
  • 20 of 13385 strict pairs visualised
  • Country borders overlay for geographic context
  • Quick zoom sidebar for navigation
  • Individual zoom buttons in popups
  • Multiple basemap options
  • All pairs meet ≤1° angle difference requirement


## End of workflow